# This Demo Uses the BRICS QUERY-API tool to: 
- [I) Find all studies in a BRICS instance (FITBIR) that use two or more common forms of interest](#Query-I)
- [II) Extract the multi-study dataset corresponding just to those studies using ALL the forms of interest](#Query-II)


### For More Information regarding FITBIR Query-API endpoints please see 
https://fitbir.nih.gov/gateway/query-api/swagger-ui/index.html 


### General Set-up: 

#### Import Dependencies

In [6]:
#import dependencies
import matplotlib.pyplot as plt
import requests
import getpass
import maskpass
from io import StringIO
import json
import pandas as pd

 #### Enter Query-API Authentication Token (log into FITBIR repository account and click under Account Management) 

In [7]:
#Set Input token: Tokens can be obtained by logging into your FITBIR repository account and clicking on Account Management tab

#token = maskpass.askpass(prompt="Enter your user token")
token = getpass.getpass('Enter your user token:')

Enter your user token:········


In [ ]:
# The token expires after 30 minutes: to renew the token 
#urlRenew = 'https://fitbir.nih.gov/gateway/rasauth/user/extendApi'

#renew = requests.get(urlRenew, headers={'Authorization':'Bearer ' + token})
#token = renew.text

### I) Find all studies in a BRICS instance (FITBIR) that use two or more common forms of interest <br/> <a name="Query-I"></a>

Interactively Search All FITBIR Forms via the web-interface here: https://fitbir.nih.gov/content/data-dictionary#form-structures

#### Set up the Request Header 

For more specifications regarding this endpoint please see: https://fitbir.nih.gov/gateway/query-api/swagger-ui/index.html#/study/getStudyByFormName <br />

#### Define Desired Search Parameters

In [8]:
# Input BRICS instance: 
BRICSInstance = 'fitbir.nih.gov'

# Input two or more short form names of interest 
#formNames = [ "Biomarker","ImagingFunctionalMR","DemogrFITBIR"]
formNames = ["GCS", "ImagingDiffusion","DemogrFITBIR"]

#### Define the Header and Query URL

In [9]:
# Define the Header and Query URL 

formstructureheader = {
    'accept': 'application/json',
    'Content-type': 'application/json',
     'Authorization':'Bearer ' + token    
}

formstructureurl =  "https://" + BRICSInstance + "/gateway/query-api/study/form?formName="

#### Perform API Request : Find those studies using each form of interest

In [10]:
# look over API request for each form 
output = []
for formName in formNames:
    #Request GET data
    formstructurequery = requests.get(formstructureurl + formName,headers=formstructureheader)

    # check status code to make sure ran correctly 
    if formstructurequery.status_code == 200: 
        print('The API Request was run successfully')
    else: 
        print('There was an Error in your API request: Check Status Code- ' + str(formstructurequery.status_code))

    # obtain the information in json form from query output
    formstructureinformation = formstructurequery.json()
    # Convert from json to panda data frame 
    formstructuredata = pd.json_normalize(formstructureinformation,record_path ="studies")
    output.append(formstructuredata)
    print(f'{formstructurequery}: There are {len(formstructureinformation[0]["studies"])} studies with {formstructureinformation[0]["form"]} data.')




The API Request was run successfully
<Response [200]>: There are 8 studies with GCS data.
The API Request was run successfully
<Response [200]>: There are 17 studies with ImagingDiffusion data.
The API Request was run successfully
<Response [200]>: There are 47 studies with DemogrFITBIR data.


#### Find the studies that use all selected forms

In [11]:
# Find the studies that use all selected forms 
count = 0
inter =[]
for outputC in output: 
    outputC['id']
    if count ==0:
        x = set(output[count]['id'])
        y = set(output[count+1]['id'])
        inter = x.intersection(y)
    else: 
        z = set(output[count]['id'])
        inter = inter.intersection(y)
inter = list(inter)
print('There are ' + str(len(inter)) + ' studies all using the forms' + str(formNames)) 

There are 3 studies all using the forms['GCS', 'ImagingDiffusion', 'DemogrFITBIR']


In [12]:
pd.set_option('display.max_colwidth', None)
forms1 = output[0] # because we placed and intersection requirement we know these studies will be in any of the outputs
forms1 = forms1.set_index('id')
# Get the information corresponding to the intersecting 
dfIntersect = forms1.loc[inter]
toShow = dfIntersect[['title','pi']]
toShow

,title,pi
id,,
FITBIR-STUDY0000267,Transforming Research and Clinical Knowledge in Traumatic Brain Injury (TRACK-TBI) - Adult,"Geoffrey Manley, MD, PhD"
FITBIR-STUDY0000409,Transforming Research and Clinical Knowledge in Traumatic Brain Injury (TRACK-TBI) - Pediatric,"Geoffrey Manley, MD, PhD"
FITBIR-STUDY0000411,"Transforming Research and Clinical Knowledge in TBI (TRACK-TBI) - High Definition Fiber Tracking Neuroimaging, Biospecimen, and Data Informatics Repositories","David Okonkwo, MD, PhD"


### Extract the multi-study dataset corresponding to just those studies using ALL the forms of interest <a name="Query-II"></a>

In [13]:
# Define API headers and query URL

#Typically Included in Request Header 
#Accept - For specifying the type of content that can be accepted in a response. (json etc) See corresponding swagger documentation for options
#Content-type - For indicating the type of content that’s included in the request body. (json etc) See corresponding swagger documentation for details as to the format of the request body
#Authorization - in the format of 'Bearer ' + token 

headers = {
    'accept': 'application/zip', # note this cannot be 'application/csv' as it was in the old example files - it needs to match the response 
    'Content-type': 'application/json',
    'Authorization':'Bearer ' + token }

queryurl ="https://" + BRICSInstance + "/gateway/query-api/data/csv"

In [15]:
forFilter = []
for f in formNames:
    c =  {
        "form": f,
        "studies": inter
    }
    forFilter.append(c)

filterFinal = {
    "formStudy": forFilter  
}
filterFinal 

{'formStudy': [{'form': 'GCS',
   'studies': ['FITBIR-STUDY0000267',
    'FITBIR-STUDY0000409',
    'FITBIR-STUDY0000411']},
  {'form': 'ImagingDiffusion',
   'studies': ['FITBIR-STUDY0000267',
    'FITBIR-STUDY0000409',
    'FITBIR-STUDY0000411']},
  {'form': 'DemogrFITBIR',
   'studies': ['FITBIR-STUDY0000267',
    'FITBIR-STUDY0000409',
    'FITBIR-STUDY0000411']}]}

In [16]:
#Post filter to Query API- 
%time query = requests.post(queryurl,headers=headers,json=filterFinal)

# check status code to make sure ran correctly 
if query.status_code == 200: 
    print('The API Request was run successfully!')
else: 
    print('There was an Error in your API request: Check Status Code- ' + str(query.status_code) + 'for details')

CPU times: total: 1.95 s
Wall time: 53.3 s
The API Request was run successfully!


In [17]:
print(f'Data received: {query.headers["Content-Disposition"][21:96]}') 

Data received: query_result_GCS_join_ImagingDi_join_DemogrFIT_2024-04-10T09-07-39808638042


In [18]:
# Retrieve the data from the query and reformat from text to pandas dataframe
dataset = query.text
texttodf = StringIO(dataset)

fitbir_data = pd.read_csv(texttodf, sep=",",low_memory=False)

pd.set_option('display.max_columns', None)
fitbir_data

,GUID,GCS.Study ID,GCS.Dataset,GCS.Main.GUID,GCS.Main.ASSOCIATED GUID,GCS.Main.SubjectIDNum,GCS.Main.AgeYrs,GCS.Main.VitStatus,GCS.Main.VisitDate,GCS.Main.SiteName,GCS.Main.DaysSinceBaseline,GCS.Main.CaseContrlInd,GCS.Main.GeneralNotesTxt,GCS.Form Administration.ContextType,GCS.Form Administration.ContextTypeOTH,GCS.Form Administration.DataSource,GCS.Form Administration.DataSourceOTH,GCS.Glasgow Coma Scale.GCSConfounderTyp,GCS.Glasgow Coma Scale.GCSEyeRespnsScale,GCS.Glasgow Coma Scale.GCSMotorRespnsScale,GCS.Glasgow Coma Scale.GCSVerbalRspnsScale,GCS.Glasgow Coma Scale.GCSTotalScore,GCS.Glasgow Coma Scale.PupilReactivityLghtLftEyeReslt,GCS.Glasgow Coma Scale.PupilReactivityLghtRtEyeReslt,GCS.Glasgow Coma Scale.PupilLftEyeMeasr,GCS.Glasgow Coma Scale.PupilRtEyeMeasr,GCS.Glasgow Coma Scale.PupilShapeLftEyeTyp,GCS.Glasgow Coma Scale.PupilShapeRtEyeTyp,GCS.Pediatric Glasgow Coma Scale.GCSConfounderTyp,GCS.Pediatric Glasgow Coma Scale.PGCSEyeRespnsScore,GCS.Pediatric Glasgow Coma Scale.PGCSMotorRespnsScore,GCS.Pediatric Glasgow Coma Scale.PGCSVerbalRespnsScore,GCS.Pediatric Glasgow Coma Scale.PGCSTotalScore,GCS.Pediatric Glasgow Coma Scale.PupilReactivityLghtLftEyeReslt,GCS.Pediatric Glasgow Coma Scale.PupilReactivityLghtRtEyeReslt,GCS.Pediatric Glasgow Coma Scale.PupilLftEyeMeasr,GCS.Pediatric Glasgow Coma Scale.PupilRtEyeMeasr,GCS.Pediatric Glasgow Coma Scale.PupilShapeLftEyeTyp,GCS.Pediatric Glasgow Coma Scale.PupilShapeRtEyeTyp,ImagingDiffusion.Study ID,ImagingDiffusion.Dataset,ImagingDiffusion.Main.GUID,ImagingDiffusion.Main.ASSOCIATED GUID,ImagingDiffusion.Main.SubjectIDNum,ImagingDiffusion.Main.AgeYrs,ImagingDiffusion.Main.VisitDate,ImagingDiffusion.Main.SiteName,ImagingDiffusion.Main.DaysSinceBaseline,ImagingDiffusion.Main.CaseContrlInd,ImagingDiffusion.Main.GeneralNotesTxt,ImagingDiffusion.Image Information.ImgStdyDateTime,ImagingDiffusion.Image Information.ImgFile,ImagingDiffusion.Image Information.ImgFileFormatTyp,ImagingDiffusion.Image Information.ImgFileFormatTypOTH,ImagingDiffusion.Image Information.ImgModltyTyp,ImagingDiffusion.Image Information.ImgModltyOTH,ImagingDiffusion.Image Information.ImgPreviewFile,ImagingDiffusion.Image Information.ImgFileHashCode,ImagingDiffusion.Image Information.ImgAcquisitionDur,ImagingDiffusion.Image Information.ImgAntmicSite,ImagingDiffusion.Image Information.ImgAntmicSitOTH,ImagingDiffusion.Image Information.ImgHeadPostnTxt,ImagingDiffusion.Image Information.ImgContrastAgentUsedInd,ImagingDiffusion.Image Information.ImgContrastAgentName,ImagingDiffusion.Image Information.ImgContrastAgentDose,ImagingDiffusion.Image Information.ImgContrastAgentInjctnTime,ImagingDiffusion.Image Information.ImgContrastAgentInjctnRate,ImagingDiffusion.Image Information.ImgContrastAgentMethodTyp,ImagingDiffusion.Image Information.ImgContrastAgentMethodOTH,ImagingDiffusion.Image Information.SedationUsedInd,ImagingDiffusion.Image Information.ImgPrescnProcdrTyp,ImagingDiffusion.Image Information.ImgPrescnProcdrOTH,ImagingDiffusion.Image Information.ImgScannerManufName,ImagingDiffusion.Image Information.ImgScannerManufOTH,ImagingDiffusion.Image Information.ImgScannerModelName,ImagingDiffusion.Image Information.ImgScannerModelOTH,ImagingDiffusion.Image Information.ImgScannerSftwrName,ImagingDiffusion.Image Information.ImgScannerSftwrVrsnNum,ImagingDiffusion.Image Information.ImgProvenanceTxt,ImagingDiffusion.Image Information.ImgAnlysisSftwrName,ImagingDiffusion.Image Information.ImgAnlysisSftwrOTH,ImagingDiffusion.Image Information.ImgAnlysisSftwrVrsnNum,ImagingDiffusion.Image Information.ImgNotesTxt,ImagingDiffusion.Image Information.ImgSklStrpPerfInd,ImagingDiffusion.Image pixel information and dimensions.ImgDimensionTyp,ImagingDiffusion.Image pixel information and dimensions.ImgDimensionOTH,ImagingDiffusion.Image pixel information and dimensions.ImgDim1ExtentVal,ImagingDiffusion.Image pixel information and dimensions.ImgDim1ResolVal,ImagingDiffusion.Image pixel information and dimensions.ImgDim1UoMVal,Imag

In [19]:
fitbir_data['GCS.Study ID'].unique()

array([411., 267., 409.,  nan])